# Import libraries

Import Numpy and cv2 to enable using OpenCV.

In [1]:
import numpy as np
import cv2

# Define the functions

Get the background first using background subtraction technique which is then to be subtracted using frame differencing technique. Please see the comments inside the functions below for the description in detail. 

In [2]:
def get_background(file_path):
    """
    Get the median frame as the background frame.
    
    Parameters
    ----------
    file_path : str
    The file path in string.
    
    Return
    ----------
    median_frame : NumPy array
    The median frame.
    
    """   
    cap = cv2.VideoCapture(file_path)
    
    # randomly select 50 frames for the calculating the median
    frame_indices = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=50)
    
    # store the frames in array
    frames = []
    for idx in frame_indices:
        # set the frame id to read that particular frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        frames.append(frame)

    # calculate the median
    median_frame = np.median(frames, axis=0).astype(np.uint8)
    
    return median_frame

def initialize(file_path):
    """
    Initialize the setting. 
    Get the background frame and read the video file.
    
    Parameters
    ----------
    file_path : str
    The file path in string.
    
    Return
    ----------
    background, cap : NumPy array, cv2.VideoCapture
    The background frame and the video capture.
    
    """   
    # get the background frame
    background = get_background(file_path)

    # convert the background model to grayscale format
    background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    
    # Read the video file
    cap = cv2.VideoCapture(file_path)

    return background, cap


def detect_cars(background, cap):
    """
    Detect cars by drawing contour of each car.
    
    Parameters
    ----------
    background : NumPy array
    The background frame.

    cap : cv2.VideoCapture
    The video capture.
    
    """       
    # the number of consecutive frames per process 
    consecutive_frame = 8
    
    frame_count = 0
    frame_diff_list = []

    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frame_count += 1
            orig_frame = frame.copy()

            # convert the frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # initialize frame difference list 
            if frame_count % consecutive_frame == 0 or frame_count == 1:
                frame_diff_list = []

            # find the difference between current frame and base frame
            frame_diff = cv2.absdiff(gray, background)

            # thresholding to convert the frame to binary
            ret, thres = cv2.threshold(frame_diff, 50, 255, cv2.THRESH_BINARY)

            # dilate the frame a bit to get some more white area
            # makes the detection of contours a bit easier
            dilate_frame = cv2.dilate(thres, None, iterations=2)

            # append the final result into the `frame_diff_list`
            frame_diff_list.append(dilate_frame)

            # if we have reached `consecutive_frame` number of frames
            if len(frame_diff_list) == consecutive_frame:

                # add all the frames in the `frame_diff_list`
                sum_frames = sum(frame_diff_list)

                # find the contours around the white segmented areas
                contours, hierarchy = cv2.findContours(sum_frames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)            

                for contour in contours:

                    # skip if contour area size is human           
                    if cv2.contourArea(contour) < 3000:
                        continue

                    # get coordinates from the contours
                    (x, y, w, h) = cv2.boundingRect(contour)

                    # draw the bounding boxes
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

                cv2.imshow('Detected Cars', frame)
                #out.write(orig_frame)

            # Stop the program by pressing 'q'   
            if cv2.waitKey(1) == ord('q'):
                break
        else:
            break

    # destroy all the windows after the loop release the video object
    cap.release()
    cv2.destroyAllWindows()

# Detect moving cars

Run below cell to start and press 'q' to stop the application.

In [3]:
background, cap = initialize('Traffic_Laramie_1.mp4')

detect_cars(background, cap)